In [1]:
import numpy as np
import pandas as pd
import re

import tensorflow as tf
from tensorflow.keras.layers import Embedding, Dense, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
test= pd.read_csv('/content/SMS_test (2).csv',encoding='latin1')
train= pd.read_csv('/content/SMS_train (1).csv',encoding='latin1')

In [3]:
test

,S. No.,Message_body,Label
0,1,"UpgrdCentre Orange customer, you may now claim...",Spam
1,2,"Loan for any purpose £500 - £75,000. Homeowner...",Spam
2,3,Congrats! Nokia 3650 video camera phone is you...,Spam
3,4,URGENT! Your Mobile number has been awarded wi...,Spam
4,5,Someone has contacted our dating service and e...,Spam
...,...,...,...
120,121,7 wonders in My WORLD 7th You 6th Ur style 5th...,Non-Spam
121,122,Try to do something dear. You read something f...,Non-Spam
122,123,Sun ah... Thk mayb can if dun have anythin on....,Non-Spam
123,124,"SYMPTOMS when U are in love: ""1.U like listeni...",Non-Spam


In [4]:
train

,S. No.,Message_body,Label
0,1,Rofl. Its true to its name,Non-Spam
1,2,The guy did some bitching but I acted like i'd...,Non-Spam
2,3,"Pity, * was in mood for that. So...any other s...",Non-Spam
3,4,Will ü b going to esplanade fr home?,Non-Spam
4,5,This is the 2nd time we have tried 2 contact u...,Spam
...,...,...,...
952,953,hows my favourite person today? r u workin har...,Non-Spam
953,954,How much you got for cleaning,Non-Spam
954,955,Sorry da. I gone mad so many pending works wha...,Non-Spam
955,956,Wat time ü finish?,Non-Spam


In [5]:
train.drop('S. No.',axis=1,inplace=True)
test.drop('S. No.',axis=1,inplace=True)

In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 957 entries, 0 to 956
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Message_body  957 non-null    object
 1   Label         957 non-null    object
dtypes: object(2)
memory usage: 15.1+ KB


In [7]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125 entries, 0 to 124
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Message_body  125 non-null    object
 1   Label         125 non-null    object
dtypes: object(2)
memory usage: 2.1+ KB


In [8]:
train.columns=['text','label']
test.columns=['text','label']

In [9]:
train['label']=train['label'].replace(['Spam','Non-Spam'],[1,0])
test['label']=test['label'].replace(['Spam','Non-Spam'],[1,0])

In [ ]:
train

,text,label
0,Rofl. Its true to its name,0
1,The guy did some bitching but I acted like i'd...,0
2,"Pity, * was in mood for that. So...any other s...",0
3,Will ü b going to esplanade fr home?,0
4,This is the 2nd time we have tried 2 contact u...,1
...,...,...
952,hows my favourite person today? r u workin har...,0
953,How much you got for cleaning,0
954,Sorry da. I gone mad so many pending works wha...,0
955,Wat time ü finish?,0


In [10]:
import string
def clean_text(text):
     # Remove special characters and numbers
    text = re.sub(r'[^A-Za-zÀ-ú ]+', '', text)

    # Convert to lower case
    text = text.lower()

    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    return text

train['text'] = train['text'].apply(clean_text)
test['text'] = test['text'].apply(clean_text)

In [ ]:
train

,text,label
0,rofl its true to its name,0
1,the guy did some bitching but i acted like id ...,0
2,pity was in mood for that soany other suggestions,0
3,will b going to esplanade fr home,0
4,this is the nd time we have tried contact u u ...,1
...,...,...
952,hows my favourite person today r u workin hard...,0
953,how much you got for cleaning,0
954,sorry da i gone mad so many pending works what...,0
955,wat time finish,0


In [29]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer, SnowballStemmer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [12]:
def remove_stopwords(texto):
    stop_words = set(stopwords.words('english'))
    tokens = nltk.word_tokenize(texto.lower())
    return " ".join([token for token in tokens if token not in stop_words])


train['text'] = train['text'].apply(remove_stopwords)
test['text'] = test['text'].apply(remove_stopwords)

In [13]:
def normalize_text(text):
    stemmer = SnowballStemmer("english")
    normalized_text = []
    for word in text.split():
        stemmed_word = stemmer.stem(word)
        normalized_text.append(stemmed_word)

    return ' '.join(normalized_text)


train['text'] = train['text'].apply(normalize_text)
test['text'] = test['text'].apply(normalize_text)

In [14]:
train

,text,label
0,rofl true name,0
1,guy bitch act like id interest buy someth els ...,0
2,piti mood soani suggest,0
3,b go esplanad fr home,0
4,nd time tri contact u u pound prize claim easi...,1
...,...,...
952,how favourit person today r u workin hard coul...,0
953,much got clean,0
954,sorri da gone mad mani pend work,0
955,wat time finish,0


In [15]:
# Maximum number of words to be considered in the vocabulary
max_words = 10000
# Maximum number of tokens in a sequence
max_len = 200

tokenizer = Tokenizer(num_words = max_words)
tokenizer.fit_on_texts(train['text'])
# Converts texts into strings of numbers
sequences_train = tokenizer.texts_to_sequences(train['text'])
sequences_val = tokenizer.texts_to_sequences(test['text'])
# Mapping words to indexes
word_index = tokenizer.word_index

In [16]:
data_train = pad_sequences(sequences_train, maxlen = max_len)
data_val = pad_sequences(sequences_val, maxlen = max_len)

In [35]:
model = tf.keras.Sequential()
model.add(Embedding(max_words, 16, input_length = max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(32, activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid'))

model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])


In [22]:
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 200, 16)           160000    
                                                                 
 global_average_pooling1d_1  (None, 16)                0         
  (GlobalAveragePooling1D)                                       
                                                                 
 dense_1 (Dense)             (None, 32)                544       
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 160577 (627.25 KB)
Trainable params: 160577 (627.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [23]:
history = model.fit(data_train, train['label'], epochs = 15, batch_size = 64, validation_data = (data_val, test['label']))

Epoch 1/15
15/15 [==============================] - 4s 179ms/step - loss: 0.6633 - accuracy: 0.8725 - val_loss: 0.7105 - val_accuracy: 0.3920
Epoch 2/15
15/15 [==============================] - 1s 88ms/step - loss: 0.6201 - accuracy: 0.8725 - val_loss: 0.7342 - val_accuracy: 0.3920
Epoch 3/15
15/15 [==============================] - 1s 57ms/step - loss: 0.5662 - accuracy: 0.8725 - val_loss: 0.7754 - val_accuracy: 0.3920
Epoch 4/15
15/15 [==============================] - 1s 74ms/step - loss: 0.5071 - accuracy: 0.8725 - val_loss: 0.8424 - val_accuracy: 0.3920
Epoch 5/15
15/15 [==============================] - 1s 57ms/step - loss: 0.4504 - accuracy: 0.8725 - val_loss: 0.9393 - val_accuracy: 0.3920
Epoch 6/15
15/15 [==============================] - 1s 71ms/step - loss: 0.4068 - accuracy: 0.8725 - val_loss: 1.0604 - val_accuracy: 0.3920
Epoch 7/15
15/15 [==============================] - 1s 63ms/step - loss: 0.3823 - accuracy: 0.8725 - val_loss: 1.1798 - val_accuracy: 0.3920
Epoch 8/15
1

In [25]:
model.save('/content/nlp.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [28]:
from tensorflow.keras.models import load_model
model = load_model('/content/nlp.h5')

In [34]:
def predict_text(text):
    # Apply the same preprocessing steps as during training
    text = clean_text(text)
    text = remove_stopwords(text)
    text = normalize_text(text)

    sequence = tokenizer.texts_to_sequences([text])
    padded_sequence = pad_sequences(sequence, maxlen=max_len)

    prediction = model.predict(padded_sequence)[0][0]

    return prediction


text_to_predict = "This is the 2nd time we have tried 2 contact u. U have won the �750 Pound prize. 2 claim is easy, call 087187272008 NOW1! Only 10p per minute. BT-national-rate."
prediction = predict_text(text_to_predict)
print(f"Predicted Probability: {prediction}")

classification = 'spam' if prediction >= 0.5 else 'non spam'
print(f"Class: {classification}")

1/1 [==============================] - 0s 50ms/step
Predicted Probability: 0.1365552693605423
Class: non spam
